In [1]:
import os
from supabase import Client, create_client
from dotenv import load_dotenv
from pathlib import Path
import json
import pydantic

In [ ]:
PROJECTDIR = Path("/home/akash/Main/projects/CACourses")
load_dotenv(PROJECTDIR / "backend/.env")

SUPA_URL: str | None = os.getenv("SUPABASE_URL")
SUPA_KEY: str | None = os.getenv("SUPABASE_ANON_KEY")

if not (SUPA_URL and SUPA_KEY):
    raise RuntimeError("Could not find environment variables SUPA_URL or SUPA_KEY.")

SUPA_CLIENT: Client = create_client(supabase_url=SUPA_URL, supabase_key=SUPA_KEY)
del SUPA_URL, SUPA_KEY

In [ ]:
from pydantic import BaseModel


class ArticulationRow(BaseModel):
    cc: int
    articulation: str

class QueryResponse(BaseModel):
    data: list[ArticulationRow]

In [11]:
import json
import os
from postgrest.exceptions import APIError
from typing import Any

# Initialize client outside the handler to reuse connection across "warm" starts
# SUPA_CLIENT = ... 

def create_response(status_code: int, body: Any):
    return {
        "statusCode": status_code,
        "headers": {"Content-Type": "application/json"},
        "body": json.dumps(body)
    }

def get_articulations(course_id: int):
    try:
        query = (
            SUPA_CLIENT
            .table("articulations")
            .select("cc", "articulation")
            .eq("course_id", course_id)
            .execute()
        )
        
        # Transform data into the desired dictionary format
        articulation_map: dict[int, str] = {
            elem.get("cc"): elem.get("articulation") 
            for elem in query.data 
            if isinstance(elem, dict)
        } # type: ignore

        course_id_set = set()
        for articulation_str in articulation_map.values():
            articulation = json.loads(articulation_str)
            for and_group in articulation.get("items"):
                course_id_set.update(and_group.get("items"))
        
        query = (
            SUPA_CLIENT
            .table("glossary")
            .select("*")
            .in_("course_id", course_id_set)
            .execute()
        )
        
        glossary_map: dict[int, str] = {
            elem.get("course_id"): elem
            for elem in query.data
            if isinstance(elem, dict)
        } # type: ignore

        return create_response(200, [articulation_map, glossary_map])
        
    except APIError as e:
        print(f"Database error: {e}") # Log for CloudWatch
        return create_response(502, {"error": "Database connection failed"})
    except Exception as e:
        print(f"Unexpected error: {e}")
        return create_response(500, {"error": "Internal server error"})
    

def lambda_handler(event, context):
    params = event.get('queryStringParameters') or {}

    # 1. Validation: Check existence
    if not (course_id_raw := params.get("course_id")) :
        return create_response(400, {"message": "Missing course_id parameter"})

    # 2. Validation: Check type
    try:
        course_id = int(course_id_raw)
    except ValueError:
        return create_response(400, {"message": "course_id must be an integer"})

    # 3. 
    return get_articulations(course_id)
    

def call_lambda(course_id: int):
    event = {"queryStringParameters": {"course_id": course_id}}
    return lambda_handler(event=event, context=None)

In [12]:
data = call_lambda(281516)
json.loads(data.get("body"))

[{'3': '{"conj":"Or","items":[{"conj":"And","items":[253995,290792]}]}',
  '4': '{"conj":"Or","items":[{"conj":"And","items":[50726,271684]}]}',
  '5': '{"conj":"Or","items":[{"conj":"And","items":[44471,254073]}]}',
  '6': '{"conj":"Or","items":[{"conj":"And","items":[280776]}]}',
  '10': '{"conj":"Or","items":[{"conj":"And","items":[299709,279985]}]}',
  '14': '{"conj":"Or","items":[{"conj":"And","items":[355210]}]}',
  '16': '{"conj":"Or","items":[{"conj":"And","items":[308610]}]}',
  '28': '{"conj":"Or","items":[{"conj":"And","items":[353439,353588]}]}',
  '30': '{"conj":"Or","items":[{"conj":"And","items":[378806]}]}',
  '33': '{"conj":"Or","items":[{"conj":"And","items":[216360,3778,40792]}]}',
  '40': '{"conj":"Or","items":[{"conj":"And","items":[277052,246462]}]}',
  '41': '{"conj":"Or","items":[{"conj":"And","items":[303064]}]}',
  '43': '{"conj":"Or","items":[{"conj":"And","items":[359154,353483,280570]},{"conj":"And","items":[284340,353483,280570]}]}',
  '44': '{"conj":"Or",

In [ ]:
d = [tuple(row.values()) for row in data.data]

In [ ]:
d